# Kafka consume

In [ ]:
from confluent_kafka import Consumer, KafkaException
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer
from confluent_kafka.serialization import SerializationContext, MessageField
from typing import List, Dict

KAFKA_CONFIG = {
    "bootstrap.servers": "localhost:9092",
    "group.id": "notebook-consumer-group",
    "auto.offset.reset": "earliest",
    "enable.auto.commit": False,
}

SCHEMA_CONFIG = {"url": "http://localhost:8081"}
schema_registry_client = SchemaRegistryClient(SCHEMA_CONFIG)
avro_deserializer = AvroDeserializer(schema_registry_client)


def consume_kafka_messages(topic: str, max_messages: int) -> List[Dict]:
    consumer = Consumer(KAFKA_CONFIG)
    consumer.subscribe([topic])

    messages = []
    print(f"Subscribed to topic: {topic}")
    print(f"Waiting for up to {max_messages} Avro-encoded messages...")

    try:
        while len(messages) < max_messages:
            msg = consumer.poll(timeout=2.0)

            if msg is None:
                print("No message received.")
                continue
            if msg.error():
                raise KafkaException(msg.error())

            value = avro_deserializer(
                msg.value(), SerializationContext(topic, MessageField.VALUE)
            )

            if value is not None:
                messages.append(value)
                print(f"Received message {len(messages)} of {max_messages}")
                consumer.commit(msg)

    finally:
        consumer.close()

    return messages


messages = consume_kafka_messages("storefront.public.orders", 5)
messages

Subscribed to topic: storefront.public.orders
Waiting for up to 5 Avro-encoded messages...
No message received.
Received message 1 of 5
Received message 2 of 5
Received message 3 of 5
Received message 4 of 5
Received message 5 of 5


[{'before': None,
  'after': {'id': 7,
   'customer_id': 7,
   'status': 'pending',
   'total': Decimal('2770.64'),
   'created_at': 1742973185794302},
  'source': {'version': '3.0.8.Final',
   'connector': 'postgresql',
   'name': 'storefront',
   'ts_ms': 1742994813553,
   'snapshot': 'true',
   'db': 'storefront',
   'sequence': '[null,"26838336"]',
   'ts_us': 1742994813553780,
   'ts_ns': 1742994813553780000,
   'schema': 'public',
   'table': 'orders',
   'txId': 800,
   'lsn': 26838336,
   'xmin': None},
  'transaction': None,
  'op': 'r',
  'ts_ms': 1742994815695,
  'ts_us': 1742994815695499,
  'ts_ns': 1742994815695499213},
 {'before': None,
  'after': {'id': 8,
   'customer_id': 8,
   'status': 'shipped',
   'total': Decimal('1940.91'),
   'created_at': 1742973186816395},
  'source': {'version': '3.0.8.Final',
   'connector': 'postgresql',
   'name': 'storefront',
   'ts_ms': 1742994813553,
   'snapshot': 'true',
   'db': 'storefront',
   'sequence': '[null,"26838336"]',
   '

In [2]:
messages[0]["after"]

{'id': 7,
 'customer_id': 7,
 'status': 'pending',
 'total': Decimal('2770.64'),
 'created_at': 1742973185794302}